In [ ]:
WINDOW_SIZE = 300  # 3 second of data (300 * 10ms)
STEP_SIZE = 50     # Slide window by 500ms for more training samples
RANDOM_SEED = 42

In [38]:
import pandas as pd

def load_csv(filename):
    df = pd.read_csv(filename)
    return df['intensity'].values.astype('float32')

background = load_csv('/content/data/background.csv')
distractor = load_csv('/content/data/distractor.csv')
target = load_csv('/content/data/target.csv')

print(f"Loaded {len(background)} background samples")
print(f"Loaded {len(distractor)} distractor samples")
print(f"Loaded {len(target)} target samples")

Loaded 1082 background samples
Loaded 1082 distractor samples
Loaded 12756 target samples


In [ ]:
import numpy as np

def create_windows(data, label, window_size, step):
    windows = []
    for i in range(0, len(data) - window_size, step):
        windows.append(data[i:i+window_size])
    return np.array(windows), np.array([label] * len(windows))

# Negative, no pests
x_bg, y_bg = create_windows(background, 0, WINDOW_SIZE, STEP_SIZE)
x_dist, y_dist = create_windows(distractor, 0, WINDOW_SIZE, STEP_SIZE)

# Raw target, pest sounds
x_target_clean, y_target_clean = create_windows(target, 1, WINDOW_SIZE, STEP_SIZE)

# Mix target and background
x_target_noisy = []
for win in x_target_clean:
    # Pick a random slice of background to overlay
    start = np.random.randint(0, len(distractor) - WINDOW_SIZE)
    dist_slice = distractor[start:start+WINDOW_SIZE]

    # Mix (pests + 30% Footstep)
    mixed = np.clip(win + (dist_slice * 0.3), 0, 100)
    x_target_noisy.append(mixed)

x_target_noisy = np.array(x_target_noisy)
y_target_noisy = np.array([1] * len(x_target_noisy))

# Combine everything
X = np.concatenate([x_bg, x_dist, x_target_clean, x_target_noisy])
y = np.concatenate([y_bg, y_dist, y_target_clean, y_target_noisy])

print(f"Total dataset size: {len(X)} samples")

Total dataset size: 532 samples


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

# Normalize to 0.0 - 1.0
X_train = X_train / 100.0
X_test = X_test / 100.0

In [ ]:
from sklearn.utils import class_weight
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(WINDOW_SIZE,)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') # probability 0 to 1
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = {i: weights[i] for i in range(len(weights))}

# Add extra weight to silence to force it to zero
# This makes silence two times more important than rat sounds
class_weights[0] = class_weights[0] * 2.0 

history = model.fit(
    X_train, y_train, 
    epochs=50, 
    batch_size=16, 
    validation_data=(X_test, y_test),
    class_weight=class_weights
)

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 16)             │         4,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,961 (19.38 KB)

 Trainable params: 4,961 (19.38 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.6969 - loss: 0.9500 - val_accuracy: 0.9533 - val_loss: 0.5544
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9426 - loss: 0.9187 - val_accuracy: 0.9439 - val_loss: 0.4460
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9429 - loss: 0.9114 - val_accuracy: 0.9439 - val_loss: 0.3834
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9553 - loss: 0.5702 - val_accuracy: 0.9439 - val_loss: 0.3333
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9383 - loss: 0.5742 - val_accuracy: 0.9439 - val_loss: 0.2889
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9433 - loss: 0.3425 - val_accuracy: 0.9533 - val_loss: 0.2590
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9437 - loss: 0.3648 - val_accuracy: 0.9533 - val_loss: 0.2173
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9454 - loss: 0.4758 - val_accuracy: 0.9533 - val_loss

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

print(f"Model size: {len(tflite_model)} bytes")

Saved artifact at '/tmp/tmpfr1cqsdq'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 300), dtype=tf.float32, name='keras_tensor_35')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  139625494327376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139625630201424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139625630202192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139625630197200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139625630203152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139625624865296: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model size: 21928 bytes


In [43]:
def hex_to_c_array(hex_data, var_name):
    c_str = f"unsigned char {var_name}[] = {{\n  "
    for i, val in enumerate(hex_data):
        c_str += f"0x{val:02x}, "
        if (i + 1) % 12 == 0:
            c_str += "\n  "
    c_str += "\n};\n"
    c_str += f"unsigned int {var_name}_len = {len(hex_data)};"
    return c_str

with open("/content/src/rat_model.h", "w") as f:
    f.write(hex_to_c_array(tflite_model, "rat_model_tflite"))

print("Saved rat_model.h")

Saved rat_model.h
